In [1]:
##############
#IMPORTS
%reset

import pickle
import numpy as np
import copy 
from sklearn import decomposition
import os
import matplotlib.pyplot as plt
import yaml
from keras import models
from keras import layers
from keras import optimizers
import tensorflow as tf
from helpers import helper_functions

def load_obj(name):
    with open(name+'.pkl', 'rb') as f:
        return pickle.load(f, encoding='latin1')

# Gaussian normalization, return 0 if std is 0
def normalize(obj, mean, std):
    a = obj-mean
    b=std
    return np.divide(a, b, out=np.zeros_like(a), where=b!=0)

# load conf file, 'config'
def load_config(config_file):
    with open(config_file) as f:
        config = yaml.load(f)
    return config

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


Using TensorFlow backend.


In [2]:
root_dir="/global/cscratch1/sd/al34/autoruns/data_50ms_trying_different_predictions/predicting_dens_smaller_archi"

input_dir = "/global/cscratch1/sd/al34/data_50ms/pred_dens_half_data_1_inputs_roryjoe"


In [5]:
# read in model and conf file
model_filename='model.h5'


config=load_config(os.path.join(root_dir,'conf.yaml'))['data_and_model']

In [6]:
config

{'delay': 2,
 'lookback': 4,
 'sigs_0d': ['curr', 'pinj', 'ech'],
 'sigs_1d': ['dens', 'temp'],
 'sigs_predict': ['dens']}

In [7]:

###############
#LOAD DATA




# # load raw data, 'raw_data'
# with open(os.path.join(input_dir,'final_data.pkl'), 'rb') as f: 
#         raw_data=pickle.load(f, encoding='latin1')
        

# load processed train data, 'train_data'
with open(os.path.join(input_dir,'train_data.pkl'), 'rb') as f: 
        train_data=pickle.load(f, encoding='latin1')
        
        
# load processed val data, 'val_data'
with open(os.path.join(input_dir,'val_data.pkl'), 'rb') as f: 
        val_data=pickle.load(f, encoding='latin1')

# load processed train target, 'train_target'
with open(os.path.join(input_dir,'train_target.pkl'), 'rb') as f: 
        train_target=pickle.load(f, encoding='latin1')

# load processed val target, 'val_target'
with open(os.path.join(input_dir,'val_target.pkl'), 'rb') as f: 
        val_target=pickle.load(f, encoding='latin1')
        

# load trainshots
with open(os.path.join(input_dir,'train_shot.pkl'), 'rb') as f: 
        train_shot=pickle.load(f, encoding='latin1')

# load val shots
with open(os.path.join(input_dir,'val_shot.pkl'), 'rb') as f: 
        val_shot=pickle.load(f, encoding='latin1')

        
# load model, 'loaded_model'
loaded_model=models.load_model(os.path.join(root_dir,model_filename))

loaded_model.compile(optimizer=optimizers.RMSprop(lr=.001),
                     metrics=['mae'], loss='mse')


# load process true train times, 'train_real_times'
with open(os.path.join(input_dir,'train_time.pkl'), 'rb') as f: 
        train_real_times=pickle.load(f, encoding='latin1')
        
# load process true val times, 'val_real_times'
with open(os.path.join(input_dir,'val_time.pkl'), 'rb') as f: 
        val_real_times=pickle.load(f, encoding='latin1')
        
        
# # load standard rho points, 'rho_points'
# with open(os.path.join(input_dir,'rho_standard.pkl'), 'rb') as f: 
#         rho_points=pickle.load(f, encoding='latin1')

        



# list of 0D and 1D signals, 'sig_keys_0d' and 'sig_keys_1d'
sig_keys_0d = config['sigs_0d'] #['curr_target', 'pinj']
sig_keys_1d = config['sigs_1d'] #['e_temp']
sig_keys_predict = config['sigs_predict']

all_sigs = sig_keys_0d+sig_keys_1d

print(all_sigs)


# delay and lookback values
delay = config['delay']
lookback = config['lookback']






['curr', 'pinj', 'ech', 'dens', 'temp']


In [8]:
rho_points=np.linspace(0,1,65)
# new_data = val_data[:, :, :2]
# new_data = np.append(new_data, val_data[:, :, 2:67:4], axis = 2)
# new_data = np.append(new_data, val_data[:, :, 67::4], axis = 2)
# input_data = new_data
# output_data=val_target[:, ::4]

input_data = val_data
output_data = val_target


In [ ]:

means = helper_functions.load_obj(os.path.join(input_dir,'means'))
stds = helper_functions.load_obj(os.path.join(input_dir,'stds'))


# means['thomson_dens'] = means['thomson_dens'][::4]
# means['thomson_temp'] = means['thomson_temp'][::4]
# means['temp'] = means['temp'][::4]


# stds['thomson_dens'] = stds['thomson_dens'][::4]
# stds['thomson_temp'] = stds['thomson_temp'][::4]
# stds['temp'] = stds['temp'][::4]

In [ ]:
generated_train_dict = {}
generated_target_dict = {}


# TRAINING WORK***************************************************************************************************

# combine the actuators together    
future_actuators = np.concatenate(([self.data["future_actuators"][sig][inds, :, np.newaxis] for sig in self.sigs_0d]), axis =2)
previous_actuators = np.concatenate(([self.data["previous_actuators"][sig][inds, :, np.newaxis] for sig in self.sigs_0d]), axis =2)




#         # if you want to lump actuators together (Rory's models) and have profile sigs separate. 
#         #########################layer names: ----1d sig names, "all_actuators"

#         generated_train_dict["all_actuators"] = np.concatenate((previous_actuators, future_actuators), axis = 1)
#         # get previous profiles data
#         for index, sig in enumerate(self.sigs_1d):
#             generated_train_dict[sig] = self.data["previous_profiles"][sig][inds]

#         ##########################






# if you want to split based on time (ie previous vs future) where you mash together all the sigs 
# Joe's model#####################################
# layer names:
# ---"all_previous_sigs", "future_actuators"

# combine all profiles
prev_profiles = np.concatenate(([self.data["previous_profiles"][sig][inds] for sig in self.sigs_1d]), axis = 2)
prev_profiles = np.concatenate((prev_profiles, previous_actuators), axis = 2)

# read data into the output dictionary
generated_train_dict["all_previous_sigs"] = prev_profiles

generated_train_dict["future_actuators"] = future_actuators

###################################################








# TARGET WORK ####################################
for index, sig in enumerate(self.sigs_predict):
generated_target_dict["target_{}".format(sig)] = self.target[sig][inds]



return generated_train_dict, generated_target_dict


In [ ]:
# Get ML model performance for full
val_mae = abs(loaded_model.predict(input_data)-output_data)
avg_val_mae=np.mean(val_mae, axis=0)
std_val_mae=np.std(val_mae, axis=0)

In [ ]:
# for full
baseline_mae=abs(output_data)
avg_baseline_mae=np.mean(baseline_mae, axis=0)
std_baseline_mae=np.std(baseline_mae, axis=0)

In [ ]:
# The following must have already been run: 
# shot_indices_dictionary = helper_functions.load_obj(input_dir+'shot_indices')
# train_shots = shot_indices_dictionary['train_shot_names']
# val_shots = shot_indices_dictionary['val_shot_names']
# train_shot_inds = shot_indices_dictionary['train_shot_indices']
# val_shot_inds = shot_indices_dictionary['val_shot_indices']

def plot_timestep(timestep, train=False, normalized=True, 
                  sig_to_perturb=None):
    
    delay = config['delay']
    titlesize=20
    othersize=20
    legendsize=15
    linewidth=7
    smalllinewidth=3
    
    if(train):
        data=train_data
        target=train_target
        shot_nums=train_shot
        #shot_inds=train_shot_inds
        true_times=train_real_times
    else:
        data=val_data
        target=val_target
        shot_nums=val_shot
        #shot_inds=val_shot_inds
        true_times=val_real_times
        
#     new_data = data[:, :, :2]
#     new_data = np.append(new_data, data[:, :, 2:67:4], axis = 2)
#     new_data = np.append(new_data, data[:, :, 67::4], axis = 2)
#     data = new_data

#     # get every 4th rho point for sig predict
#     new_target = target[:, ::4]
#     target = new_target
    
    

    shot_num=shot_nums[timestep]
    
    shot_range = np.where(shot_nums == shot_num)[0]
    times = list(range(shot_range[0],shot_range[-1]+1))
#     times=list(range(shot_inds[index],shot_inds[index+1]))
    
    input_data=np.array(data)[timestep:timestep+1]
    #input_data[:,:,:3]=0
    
    pred=np.ndarray.flatten(loaded_model.predict(input_data))                      
    
    true=target[timestep]
    # note that timestep 0 will now throw an error 
    prev=target[timestep-1] #data[timestep][-1][num_sigs:]
    
    if normalized:
        pred_real=pred
        true_real=true
        prev_real=prev
    else:
        pred_real = np.multiply(pred,stds['temp'])+means['temp']
        true_real = np.multiply(true,stds['temp'])+means['temp']
        #prev_real = (np.multiply(prev,stds['thomson_temp'])+means['thomson_temp'])/1000.
        prev_real = (np.multiply(prev,stds['temp'])+means['temp'])
    
    sigs=data[timestep][-1][:len(sig_keys_0d)] #pinj, tinj, curr
    sigs_all_times=np.array(data)[times,-delay-1,:].T
    
    fig = plt.figure()
    
    baseline_error = abs(true) #abs(true-prev)
    pred_error = abs(true-pred)
    
    ax = fig.add_subplot('222')
    ax.set_title('Electron temperature predictions by the ML model', fontsize=titlesize)
    ax.plot(rho_points,pred_real,label='Predicted', linewidth=smalllinewidth)
    
    
    #### FOR SENSITIVITY ANALYSIS ####
    if sig_to_perturb is not None:
        if sig_to_perturb in sig_keys_0d:
            sig_index=sig_keys_0d.index(sig_to_perturb)
        else:
            sig_index=range(len(sig_keys_0d),input_data.shape[2])

        input_data_perturbed=input_data.copy()


        # Flatline at the previous value
        input_data_perturbed[:,:,sig_index]=input_data_perturbed[:,0,sig_index]
        # Increase linearly
        #input_data_perturbed[:,:,sig_index]*=[1+i*.1 for i in range(input_data_perturbed.shape[1])]
        # add 10%
        #input_data_perturbed[:,:,sig_index] *= 1.1



        pred_perturbed=np.ndarray.flatten(loaded_model.predict(input_data_perturbed))
        if normalized:
            pred_perturbed_real=pred_perturbed
        else: 
            if config['n_components']!='None':
                # do the processing with pca, which was already loaded
                pred_perturbed = pca.inverse_transform(pred_perturbed)
            pred_perturbed_real = np.multiply(pred_perturbed,stds['e_temp'])+means['e_temp']
        ax.plot(rho_points,pred_perturbed_real,
                label='Predicted with perturbation to {}'.format(sig_to_perturb), 
                linewidth=smalllinewidth)
    ##################################
    
    
    ax.plot(rho_points,true_real,label='True', linewidth=smalllinewidth)
    #for differences
    if normalized:
        ax.axhline(0,label='Baseline',linewidth=smalllinewidth, color='k')
    else:
        ax.plot(rho_points,prev_real,label='Previous', linewidth=smalllinewidth)
    
    ax.set_ylabel('Normalized e_temp', fontsize=othersize)
    ax.set_xlim(0,1)
    #ax.set_xlabel('Normalized rho', fontsize=othersize)
    ax.legend(loc='best', fontsize=legendsize)
    
    ax = fig.add_subplot('224')
    ax.set_title('Error in e_temp predictions', fontsize=titlesize)
    ax.plot(rho_points, pred_error, color='red', label='Model error for one example', linewidth=linewidth)
    ax.plot(rho_points, avg_val_mae, color='red', linestyle='--', label='Average prediction error', linewidth=smalllinewidth)
    ax.fill_between(rho_points,avg_val_mae-std_val_mae, avg_val_mae+std_val_mae, color='red', alpha=.2)
    ax.axhline(0,color='k')
    ax.plot(rho_points, baseline_error, color='b', label='Baseline error for one example', linewidth=linewidth)
    ax.plot(rho_points, avg_baseline_mae, color='b', linestyle='--', label='Average baseline error', linewidth=smalllinewidth)
    ax.fill_between(rho_points,avg_baseline_mae-std_baseline_mae, avg_baseline_mae+std_baseline_mae, color='blue', alpha=.2)
    ax.set_ylim(0,.25)
    ax.set_xlim(0,1)
    ax.legend(loc='best', fontsize=legendsize)
    ax.set_ylabel('Error (abs val)', fontsize=othersize)
    ax.set_xlabel('Normalized rho', fontsize=othersize)
        
        #textstr='Baseline error: %.2f \nModel error: %.3f'%(np.mean(baseline_error),np.mean(pred_error))
        #ax.text(0.05,0.1,textstr, transform=ax.transAxes)
    
    #plt.subplots_adjust(hspace=.5)

    for i in range(len(sig_keys_0d)):
        which_plot='{}2{}'.format(len(sig_keys_0d)+len(sig_keys_1d),2*i+1) 
        ax=fig.add_subplot(which_plot) #axes[i+1]
        #ax.plot(times,sigs_all_times[i])
        if normalized:
            ax.plot(true_times[times],sigs_all_times[i])
            if sig_to_perturb is not None:
                if (i==sig_index):
                    ax.plot(true_times[timestep-config['lookback']:timestep+config['delay']+1],
                        input_data_perturbed[0,:,i],
                        linewidth=10)
        else:
            normalized_sig=(sigs_all_times[i]*stds[sig_keys_0d[i]])+means[sig_keys_0d[i]]
            ax.plot(true_times[times],normalized_sig)
        ax.set_ylabel(sig_keys_0d[i], fontsize=titlesize)
        ax.axvline(true_times[timestep],color='r')
        ax.axvline(true_times[timestep+delay],color='b')
            
    for i in range(len(sig_keys_1d)):
        which_plot='{}2{}'.format(len(sig_keys_0d)+len(sig_keys_1d),2*(len(sig_keys_0d)+i)+1) 
        ax=fig.add_subplot(which_plot) #axes[i+1]
        #ax.plot(times,sigs_all_times[i])
        rho_len_in = int((data.shape[2]-len(sig_keys_0d))/len(sig_keys_1d))
        if normalized:
            sig_trace_1d = np.mean(sigs_all_times[len(sig_keys_0d)+i*rho_len_in:len(sig_keys_0d)+(i+1)*rho_len_in],axis=0)
            ax.plot(true_times[times],sig_trace_1d)
        else:
            tmp = sigs_all_times[len(sig_keys_0d)+i*rho_len_in:len(sig_keys_0d)+(i+1)*rho_len_in]
            if config['n_components']!='None':
                # do the processing with pca, which was already loaded
                tmp = pca.inverse_transform(tmp)
            # convert back by multiplying by std, adding mean
            sig_trace_1d = (stds[sig_keys_1d[i]].dot(tmp)+sum(means[sig_keys_1d[i]]))/rho_len_in
            ax.plot(true_times[times],sig_trace_1d)
        ax.set_ylabel('mean {}'.format(sig_keys_1d[i]), fontsize=titlesize)
        ax.axvline(true_times[timestep],color='r')
        ax.axvline(true_times[timestep+delay],color='b')
        if i==len(sig_keys_0d)+len(sig_keys_1d)-1:
            ax.set_xlabel('Time (ms)', fontsize=othersize)
    
    fig.suptitle('Shot {} (timestep {}) for 100ms delay, 50ms intervals. ALL RHO POINTS'.format(shot_num,timestep))
    plt.show()

In [ ]:
plt.rcParams['figure.figsize'] = [20, 15]
font={'size': 20, 'weight': 'heavy'}
plt.rc('font', **font)

In [ ]:
import random
timestep=random.randint(0,len(val_data))
# timestep=np.random.choice(val_shot_inds[:-1])+np.random.randint(10)

#random.randint(0,len(train_data))
#timestep=np.random.choice(train_shot_inds)+np.random.randint(10)

#i+=2
#timestep=val_shot_inds[val_shots.index(163174)]+i
#timestep=val_shot_inds[42]+20
#timestep=56410


shot_nums = val_shot
shot_num=shot_nums[timestep]
    
shot_range = np.where(shot_nums == shot_num)[0]
timestep = shot_range[0] + 1



plot_timestep(timestep, train=True, normalized=True, sig_to_perturb=None)


#14030